In [11]:
using Pkg
Pkg.activate("CT")
using CT

  Activating project at `~/CT_MPS_mini/CT`


In [12]:
Pkg.status()

Project CT v0.1.0
Status `~/CT_MPS_mini/CT/Project.toml`
  [c7e460c6] ArgParse v1.2.0
⌃ [6e4b80f9] BenchmarkTools v1.5.0
  [f67ccb44] HDF5 v0.17.2
⌅ [9136182c] ITensors v0.6.16
  [682c06a0] JSON v0.21.4
⌃ [33e6dc65] MKL v0.7.0
⌃ [91a5bcdd] Plots v1.40.5
⌃ [9f0aa9f4] TCIITensorConversion v0.1.4
⌃ [b261b2ec] TensorCrossInterpolation v0.9.10
⌃ [a759f4b9] TimerOutputs v0.5.24
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


In [13]:
include("global_adder_passthrough.jl")

filter_highest_tag_number (generic function with 1 method)

In [14]:
# benchmark adder_MPO
using ITensors
# initialize random state
using .CT: _initialize_basis, _initialize_vector, P_MPO, XI_MPO, I_MPO, adder_MPO, add1, power_mpo
using Random

L = 40
ancilla = 0
folded = true
seed_vec = 123457
xj = Set([1//3, 2//3])
i1 = 1
p_ctrl = 1.0
p_proj = 0.0
_maxdim = 50
_cutoff = 1e-15
seed = 123457
x0 = nothing
qubit_site, ram_phy, phy_ram, phy_list = _initialize_basis(L, ancilla, folded)
println(phy_list)
println(ram_phy)
println(phy_ram)
rng = MersenneTwister(seed_vec)
rng_vec = seed_vec === nothing ? rng : MersenneTwister(seed_vec)
# initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim);
# println(initial_state)
shift_1_3_bits, shift_1_3_amount = fraction_to_binary_shift(1, 3, L)
# initial_state = productMPS(qubit_site, [1,2,1,2,2,1,2,1,2,1,2,1,1,1,2,2])
initial_state = _initialize_vector(L, ancilla, x0, folded, qubit_site, ram_phy, phy_ram, phy_list, rng_vec, _cutoff, _maxdim);

# initialize haining adder mpo
add1_mpo=MPO(add1(i1,L,phy_ram,phy_list),qubit_site)
add1_6,add1_3=power_mpo(add1_mpo,[div(2^L,6)+1,div(2^L,3)])

# initialize tao adder mpo
carry_links, T_vec, id_vec, gate_vec = initialize_links(L, qubit_site, shift_1_3_bits, ram_phy);
initial_state_1 = copy(initial_state)
initial_state_2 = copy(initial_state)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40]
[1, 40, 2, 39, 3, 38, 4, 37, 5, 36, 6, 35, 7, 34, 8, 33, 9, 32, 10, 31, 11, 30, 12, 29, 13, 28, 14, 27, 15, 26, 16, 25, 17, 24, 18, 23, 19, 22, 20, 21]
[1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 40, 38, 36, 34, 32, 30, 28, 26, 24, 22, 20, 18, 16, 14, 12, 10, 8, 6, 4, 2]


MPS
[1] ((dim=2|id=525|"Qubit,Site,n=1"), (dim=50|id=675|"Link,l=1"))
[2] ((dim=50|id=675|"Link,l=1"), (dim=2|id=515|"Qubit,Site,n=2"), (dim=50|id=309|"Link,l=2"))
[3] ((dim=50|id=309|"Link,l=2"), (dim=2|id=592|"Qubit,Site,n=3"), (dim=50|id=499|"Link,l=3"))
[4] ((dim=50|id=499|"Link,l=3"), (dim=2|id=949|"Qubit,Site,n=4"), (dim=50|id=858|"Link,l=4"))
[5] ((dim=50|id=858|"Link,l=4"), (dim=2|id=429|"Qubit,Site,n=5"), (dim=50|id=691|"Link,l=5"))
[6] ((dim=50|id=691|"Link,l=5"), (dim=2|id=648|"Qubit,Site,n=6"), (dim=50|id=158|"Link,l=6"))
[7] ((dim=50|id=158|"Link,l=6"), (dim=2|id=98|"Qubit,Site,n=7"), (dim=50|id=35|"Link,l=7"))
[8] ((dim=50|id=35|"Link,l=7"), (dim=2|id=784|"Qubit,Site,n=8"), (dim=50|id=670|"Link,l=8"))
[9] ((dim=50|id=670|"Link,l=8"), (dim=2|id=914|"Qubit,Site,n=9"), (dim=50|id=582|"Link,l=9"))
[10] ((dim=50|id=582|"Link,l=9"), (dim=2|id=170|"Qubit,Site,n=10"), (dim=50|id=261|"Link,l=10"))
[11] ((dim=50|id=261|"Link,l=10"), (dim=2|id=81|"Qubit,Site,n=11"), (dim=50|id=613|"

In [17]:
using BenchmarkTools
BenchmarkTools.DEFAULT_PARAMETERS.samples = 100
BenchmarkTools.DEFAULT_PARAMETERS.evals = 1
BenchmarkTools.DEFAULT_PARAMETERS.seconds = 300


300

In [ ]:
@benchmark initial_state_1 = apply(add1_3,$initial_state_1)


In [ ]:
@benchmark initial_state_2 = global_adder($initial_state_2, carry_links, T_vec, id_vec, gate_vec, qubit_site, shift_1_3_bits, ram_phy)

1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8
9
10
11
1
2
3
4
5
6
7
8


BenchmarkTools.Trial: 100 samples with 1 evaluation.
 Range (min … max):  1.779 s …    2.396 s  ┊ GC (min … max): 0.62% … 0.90%
 Time  (median):     1.966 s               ┊ GC (median):    0.97%
 Time  (mean ± σ):   2.006 s ± 127.194 ms  ┊ GC (mean ± σ):  3.54% ± 4.15%

               ▆▄ ▂██             ▂     ▂                     
  ▄▄▁▁▆▁▁█▆▆█████▁███▆█▆▆▄▆▄▆▆▄▆▄██▆▄▄▄▄█▄█▁▁▁▁▆▁▁▆▄▁▄▁▁▁▁▄▄ ▄
  1.78 s         Histogram: frequency by time         2.33 s <

 Memory estimate: 542.19 MiB, allocs estimate: 11641.

In [ ]:
BenchmarkTools.DEFAULT_PARAMETERS.seconds

300.0

In [ ]:
println(contract(initial_state_2).tensor)

[-0.00110480691571687 0.0155418649932069; 0.00571791851079534 0.004980104418543361;;; 0.009144860279238302 -0.0072022567415999505; 0.011931385027502546 -0.026980481480495103;;;; 0.008050693413112257 -0.0248028333377495; -0.01967979967686029 0.0003541477742075605;;; -0.008262590691811138 -0.007797143293526244; 0.005162000302523061 -0.007797744423932455;;;;; -0.028435457601430864 0.009209724800593646; 0.009407362957522309 -0.002047164007024342;;; 0.013482887076904471 -0.001808681089785708; 0.01925986886540166 0.0032536622509154066;;;; -0.0043095948160132 -0.00973277629637734; -0.0076673813522451836 -0.025548716642632772;;; -0.007264793282231336 0.004514201723944771; -0.0014068337368698784 0.012276241718521546;;;;;; -0.025270664333178077 -0.01829109934779068; 0.02498603279500988 -0.015327247325534735;;; -0.009197450648918004 -0.016789221498001074; -0.02525007599773812 -0.027638463063837847;;;; -0.0026322380400045325 0.013501948591680453; 0.0030787127884063696 0.005984563685458448;;; -0.00

;;; -0.017319708045104017 -0.03020510132939614; 0.00015939563020948223 -0.018544344011409693;;;;; -0.005316214883181333 0.014350807666878103; 0.015882154751411297 0.005595185857642938;;; 0.0010943808898765293 -0.0034449704738489214; 0.013525399199123635 -0.009066560232347434;;;; 0.011354737599872058 0.01935537518970359; 0.014978233902052426 0.00823641896844307;;; 0.012668984291710473 0.016718398506590834; 0.014715650078593856 0.010076881452014777;;;;;;;; -0.00923265014527105 -0.012444812957232973; -0.010698701800947962 -0.016666565751693438;;; -0.025526850546621134 0.01008960346139111; -0.005334003570218601 0.012875843596256485;;;; -0.016593942009680576 0.019337869365552585; 0.01758367483717841 0.004171404139124777;;; 0.003241607618798568 -0.009596766681150882; -0.0014014312940164636 -0.013905850126157466;;;;; 0.008742595754428087 -0.011103171355856393; 0.01008191157326324 0.012062020857843086;;; -0.0008199519402866138 0.004198110153417085; 0.000194710861697635 -0.00513586989291643;;;;

In [ ]:
# The `seconds` expression helps set a rough time budget, see Manual for more explanation
@btime sin(x) setup=(x=rand()) seconds=3

  8.600 ns (0 allocations: 0 bytes)


0.7293060492501259

In [ ]:
# psi = contract(initial_state)
# println(inds(psi))
# print_nonzero_coordinates(psi)

In [ ]:
# function perm(bit_string::String, pattern::Vector{Int})
#     permuted = ['0' for _ in 1:length(bit_string)]
#     for (i, pos) in enumerate(pattern)
#         permuted[pos] = bit_string[i]
#     end
#     return join(permuted)
# end